### Projet ETL

Le projet est composé de deux fichiers 'usertable.csv' et 'VisitorLogs.csv'. L'objectif de cette base de données d'en faire une base de donnée pour qu'une équipe de marketing puisse l'utiliser.


## Implémentation bibliothèque


In [2]:
import pandas as pd
import numpy as np
from pandas.core.interchange.dataframe_protocol import DataFrame

## Extract

La table de 'userTable.csv' 

In [89]:
df_user=pd.read_csv('userTable.csv',dtype=str)
print(df_user.head().to_string())

La table de 'VIsitorLogsData.csv'

In [1]:
df_VisitorLogs=pd.read_csv('VisitorLogsData.csv')
print(df_VisitorLogs.head().to_string())

NameError: name 'pd' is not defined

## Cleaning des données
Il y'a plusieurs à transformer dans nos données.  
Dans 'VisitorLogsData.csv'. On travaille uniquement avec les données des membres inscrits, on supprime  donc toutes les lignes qui ne possède pas de UserID.  
De plus, on formate les dates aussi, on garde uniquement les lignes avec des dates et on garde le format 'YYYY-MM-DD ... '    

In [3]:
df_VisitorLogs=df_VisitorLogs.dropna(subset=['VisitDateTime'])
df_VisitorLogs=df_VisitorLogs.dropna(subset=['UserID'])

df_VisitorLogs['VisitDateTime']=pd.to_datetime(df_VisitorLogs['VisitDateTime'],errors='coerce',unit='ns')
df_VisitorLogs = df_VisitorLogs[df_VisitorLogs['VisitDateTime'].notna()]
print(df_VisitorLogs.head().to_string())

NameError: name 'df_VisitorLogs' is not defined

## Nouveau DataFrame  
On crée un noueau DataFrame final avec les différents données que l'on veut voir.

In [91]:
colonnes = ['UserID','No_of_days_Visited_7_Days','No_Of_Products_Viewed_15_Days', 'User_Vintage','Most_Viewed_product_15_Days',
            'Most_Active_OS','Recently_Viewed_Product','Pageloads_last_7_days','Clicks_last_7_days']

df_Marketing_Model=pd.DataFrame(columns=colonnes)
df_Marketing_Model['UserID']= df_VisitorLogs['UserID'].unique()
print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
1  U134616                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
2  U130784                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
3  U120983                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
4  U120287                       NaN                        

## Travail sur les 7 derniers jours 
On va travailler avec des données qui sont sur les 7  ou 15 derniers jours. 

In [92]:
date_fin = pd.Timestamp('2018-05-27')
date_debut = date_fin - pd.Timedelta(days=7)
df_7jours= df_VisitorLogs[(df_VisitorLogs['VisitDateTime'] >= date_debut) & (df_VisitorLogs['VisitDateTime'] <= date_fin)]
print(df_7jours.head().to_string())

        webClientID           VisitDateTime ProductID   UserID  Activity        Browser        OS       City Country
68    WI10000018039 2018-05-23 09:44:44.023  Pr101042  U113937     CLICK         Safari  mac os x    Dailekh   Nepal
106  WI100000319667 2018-05-25 06:01:57.465  pr100044  U111160     click  Chrome Mobile   Android  Bengaluru   India
140    WI1000001500 2018-05-21 05:28:46.221  Pr100339  U119431     click         Chrome   windows    Kolkata   India
198  WI100000506560 2018-05-25 17:03:00.226  Pr100503  U109651  PAGELOAD         Chrome   windows  Bengaluru   India
213  WI100000727623 2018-05-21 03:55:35.958  pr100544  U113103  pageload         Chrome   Windows    Chennai   India


In [93]:
date_fin = pd.Timestamp('2018-05-27')
date_debut = date_fin - pd.Timedelta(days=15)
df_15jours= df_VisitorLogs[(df_VisitorLogs['VisitDateTime'] >= date_debut) & (df_VisitorLogs['VisitDateTime'] <= date_fin)]
print(df_15jours.head().to_string())

       webClientID           VisitDateTime ProductID   UserID Activity Browser        OS     City Country
50  WI100000110130 2018-05-19 00:02:31.347  pr100340  U120983    click  Chrome   Windows   Mumbai   India
54   WI10000014616 2018-05-19 04:51:45.337  Pr100166  U120287    click  Chrome   Windows      NaN     NaN
68   WI10000018039 2018-05-23 09:44:44.023  Pr101042  U113937    CLICK  Safari  mac os x  Dailekh   Nepal
69    WI1000001200 2018-05-13 13:17:03.751  Pr101042  U115735    click  Chrome   Windows  Khammam   India
74   WI10000022937 2018-05-16 12:39:44.279  Pr101964  U103943    click  Chrome   Windows      NaN     NaN


# Nombres de visites des utilisateurs sur 7 Jours
On veut regarder combien de visiteurs a-t-on eu les 7 derniers jours pour possiblement voir l'évolution des visites du site.

In [4]:
# Compter les visites par utilisateur
visit_counts = df_7jours['UserID'].value_counts().reset_index()
visit_counts.columns = ['UserID', 'No_of_days_Visited_7_Days']
# Effectuer la jointure avec les nouvelles valeurs
updated_values = df_Marketing_Model[['UserID']].merge(visit_counts, on='UserID', how='left')
df_Marketing_Model.update(updated_values)
# df_Marketing_Model['No_of_days_Visited_7_Days']=df_Marketing_Model['No_of_days_Visited_7_Days'].fillna(0)
pd.set_option('future.no_silent_downcasting', True)
print(df_Marketing_Model.head().to_string())

NameError: name 'df_7jours' is not defined

 # Nombres de produits vues par les utilisateurs sur les derniers 15 Jours
 

In [95]:
nbProduit15Days= df_15jours.groupby('UserID')['ProductID'].count().reset_index()
nbProduit15Days.columns=['UserID','No_Of_Products_Viewed_15_Days']
updated_values = df_Marketing_Model[['UserID']].merge(nbProduit15Days, on='UserID', how='left')
df_Marketing_Model.update(updated_values)

print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
1  U134616                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
2  U130784                       NaN                           NaN          NaN                         NaN            NaN                     NaN                   NaN                NaN
3  U120983                       1.0                          32.0          NaN                         NaN            NaN                     NaN                   NaN                NaN
4  U120287                     143.0                        

# Durée du compte

In [96]:
User_Vintage= df_user[['UserID','Signup Date']]
User_Vintage['Signup Date']=pd.Timestamp('2018-05-27', tz='UTC')-pd.to_datetime(User_Vintage['Signup Date'],format='ISO8601')
User_Vintage.columns=['UserID','User_Vintage']
# print(User_Vintage['Signup Date'].head())
updated_values = df_Marketing_Model[['UserID']].merge(User_Vintage, on='UserID', how='left')
df_Marketing_Model.update(updated_values)
print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days              User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN  239 days 05:33:17.252592                         NaN            NaN                     NaN                   NaN                NaN
1  U134616                       NaN                           NaN  446 days 08:55:01.624228                         NaN            NaN                     NaN                   NaN                NaN
2  U130784                       NaN                           NaN  261 days 05:15:35.831008                         NaN            NaN                     NaN                   NaN                NaN
3  U120983                       1.0                          32.0   76 days 18:53:27.568512                         NaN            NaN                     NaN                   NaN               

C:\Users\thoma\AppData\Local\Temp\ipykernel_3192\2476486706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  User_Vintage['Signup Date']=pd.Timestamp('2018-05-27', tz='UTC')-pd.to_datetime(User_Vintage['Signup Date'],format='ISO8601')


# Produit le plus observé les 15 derniers jours par utilisateur

In [97]:
produit_counts = df_15jours.groupby(['UserID', 'ProductID']).size().reset_index(name='count')
produit_max = produit_counts.loc[produit_counts.groupby('UserID')['count'].idxmax()]
produit_max.columns=['UserID','Most_Viewed_product_15_Days','count']
updated_values = df_Marketing_Model[['UserID']].merge(produit_max[['UserID', 'Most_Viewed_product_15_Days']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['Most_Viewed_product_15_Days']=df_Marketing_Model['Most_Viewed_product_15_Days'].fillna('Product101')
print(df_Marketing_Model.head().to_string())
# print(df_Marketing_Model.isnull().sum())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days              User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN  239 days 05:33:17.252592                  Product101            NaN                     NaN                   NaN                NaN
1  U134616                       NaN                           NaN  446 days 08:55:01.624228                  Product101            NaN                     NaN                   NaN                NaN
2  U130784                       NaN                           NaN  261 days 05:15:35.831008                  Product101            NaN                     NaN                   NaN                NaN
3  U120983                       1.0                          32.0   76 days 18:53:27.568512                    pr100340            NaN                     NaN                   NaN               

# OS le plus utilisé par utilisateur

In [98]:
OS_counts = df_VisitorLogs.groupby(['UserID', 'OS']).size().reset_index(name='count')
OS_max = OS_counts.loc[OS_counts.groupby('UserID')['count'].idxmax()]
OS_max.columns=['UserID','Most_Active_OS','count']
updated_values = df_Marketing_Model[['UserID']].merge(OS_max[['UserID', 'Most_Active_OS']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days              User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN  239 days 05:33:17.252592                  Product101        Windows                     NaN                   NaN                NaN
1  U134616                       NaN                           NaN  446 days 08:55:01.624228                  Product101        Windows                     NaN                   NaN                NaN
2  U130784                       NaN                           NaN  261 days 05:15:35.831008                  Product101      Chrome OS                     NaN                   NaN                NaN
3  U120983                       1.0                          32.0   76 days 18:53:27.568512                    pr100340        Windows                     NaN                   NaN               

## Produit vu le plus récemment 

In [99]:
df_VisitorLogs.sort_values(by=['UserID','VisitDateTime'], ascending=[True,False], inplace=True)
last_product=df_VisitorLogs.groupby('UserID').first().reset_index()
last_product = last_product.rename(columns={'ProductID': 'Recently_Viewed_Product'})
updated_values=df_Marketing_Model[['UserID']].merge(last_product[['UserID','Recently_Viewed_Product']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['Recently_Viewed_Product']=df_Marketing_Model['Recently_Viewed_Product'].fillna('Product101')
print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days              User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN  239 days 05:33:17.252592                  Product101        Windows                Pr100070                   NaN                NaN
1  U134616                       NaN                           NaN  446 days 08:55:01.624228                  Product101        Windows                Pr100495                   NaN                NaN
2  U130784                       NaN                           NaN  261 days 05:15:35.831008                  Product101      Chrome OS                Pr102323                   NaN                NaN
3  U120983                       1.0                          32.0   76 days 18:53:27.568512                    pr100340        Windows                pr100340                   NaN               

## Nombre de Page généres les 7 derniers jours

In [100]:
page_load_counts=df_7jours.groupby(by='UserID')['Activity'].value_counts().loc[:, 'pageload'].reset_index(name='Pageloads_last_7_days')
updated_values = df_Marketing_Model[['UserID']].merge(page_load_counts[['UserID', 'Pageloads_last_7_days']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days              User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN  239 days 05:33:17.252592                  Product101        Windows                Pr100070                   NaN                NaN
1  U134616                       NaN                           NaN  446 days 08:55:01.624228                  Product101        Windows                Pr100495                   NaN                NaN
2  U130784                       NaN                           NaN  261 days 05:15:35.831008                  Product101      Chrome OS                Pr102323                   NaN                NaN
3  U120983                       1.0                          32.0   76 days 18:53:27.568512                    pr100340        Windows                pr100340                   NaN               

## Nombre de clics des 7 derniers jours

In [101]:
page_load_counts=df_7jours.groupby(by='UserID')['Activity'].value_counts().loc[:, 'click'].reset_index(name='Clicks_last_7_days')
updated_values = df_Marketing_Model[['UserID']].merge(page_load_counts[['UserID', 'Clicks_last_7_days']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['Clicks_last_7_days']=df_Marketing_Model['Clicks_last_7_days'].fillna(0)
print(df_Marketing_Model.head().to_string())

    UserID No_of_days_Visited_7_Days No_Of_Products_Viewed_15_Days              User_Vintage Most_Viewed_product_15_Days Most_Active_OS Recently_Viewed_Product Pageloads_last_7_days Clicks_last_7_days
0  U132443                       NaN                           NaN  239 days 05:33:17.252592                  Product101        Windows                Pr100070                   NaN                  0
1  U134616                       NaN                           NaN  446 days 08:55:01.624228                  Product101        Windows                Pr100495                   NaN                  0
2  U130784                       NaN                           NaN  261 days 05:15:35.831008                  Product101      Chrome OS                Pr102323                   NaN                  0
3  U120983                       1.0                          32.0   76 days 18:53:27.568512                    pr100340        Windows                pr100340                   NaN               